In [ ]:
import os
import zipfile
local_zip='./umair.com.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

In [ ]:
train_horse_dir=os.path.join('./horse-or-human/horses')
train_human_dir=os.path.join('./horse-or-human/humans')

In [ ]:
train_horse_dir

In [ ]:
train_horse_name=os.listdir(train_horse_dir)
print(train_horse_name[:10])
train_human_name=os.listdir(train_human_dir)
print(train_human_name[:10])

In [ ]:
print('total training horse images:',len(os.listdir(train_horse_dir)))
print('total training human images:',len(os.listdir(train_human_dir)))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
nrow=4
ncols=4
index=0

In [ ]:
fig=plt.gcf()
fig.set_size_inches(ncols*2,nrow*2)
index +=8
next_horse_pix=[os.path.join(train_horse_dir,fname)
               for fname in train_horse_name[index-8:index]]
next_human_pix=[os.path.join(train_human_dir,fname)
               for fname in train_human_name[index-8:index]]
for i,img_path in enumerate(next_horse_pix+next_human_pix):
    sp=plt.subplot(nrow,ncols,i+1)
    img=mpimg.imread(img_path)
    plt.imshow(img)
plt.show()

In [ ]:
import tensorflow as tf

In [ ]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),  # Change lr to learning_rate
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_gen=ImageDataGenerator(rescale=1/255)
train_generator=img_gen.flow_from_directory(
    './horse-or-human/',
    target_size=(300,300),
    batch_size=128,
    class_mode='binary'
)

In [ ]:
histroy=model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1
)

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
